### Importing required libraries

In [ ]:
from torch.utils.data import DataLoader
from torchtext.datasets import IMDB
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torch 
import torch.nn as nn
from transformers import XLNetTokenizer

### Loading the dataset

In [2]:
train_iter,valid_iter=IMDB()

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
next(iter(train_iter))
next(iter(train_iter))

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

### Loading XLNetTokenizer and building vocabulary

In [7]:
tokenizer=XLNetTokenizer.from_pretrained('xlnet-base-cased')
def yield_tokens(train_iter):
    for _,data_sample in train_iter:
            yield tokenizer.tokenize(data_sample)
vocab=build_vocab_from_iterator(yield_tokens(train_iter),specials=["<unk>","<pad>",'<|endoftext|>'],special_first=True)
vocab.set_default_index(vocab['<unk>'])

In [8]:
index_to_string=vocab.get_itos()

### Making of DataLoader

In [20]:
from torchtext.vocab import build_vocab_from_iterator
import torch

# 1. Final get_sample function
def get_sample(block_size, tokens_list):
    random_start = 0
    stop = block_size
    
    # Slicing tokens
    src_sequence = tokens_list[random_start:stop]
    tgt_sequence = tokens_list[random_start + 1:stop + 1]
    
    # List conversion to avoid attribute errors and maintain alignment
    src_sequence = list(src_sequence)
    tgt_sequence = list(tgt_sequence)
    
    # Padding if target is short
    if len(tgt_sequence) < len(src_sequence):
        tgt_sequence.append('<|endoftext|>')
        
    return src_sequence, tgt_sequence

# 2. Final implementation logic
# train_iter se ek sample nikalna
label, text = next(iter(train_iter))

# IMPORTANT: .tokenize() use karein, sirf tokenizer(text) nahi
tokens = tokenizer.tokenize(text)

# Sequence generate karein
block_size = 10
src_text, tgt_text = get_sample(block_size, tokens)

# 3. Numericalization (Ab error nahi aayega kyunki ye list of strings hai)
src_indices = vocab(src_text)
tgt_indices = vocab(tgt_text)

# 4. Convert to Tensors
src_tensor = torch.tensor(src_indices, dtype=torch.long)
tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long)

print(f"Source Indices: {src_indices}")
print(f"Target Indices: {tgt_indices}")

Source Indices: [19, 1525, 19, 9566, 658, 6872, 9990, 3714, 22, 453]
Target Indices: [1525, 19, 9566, 658, 6872, 9990, 3714, 22, 453, 4556]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    a

In [21]:
BATCH_SIZE=1

batch_of_tokens=[]

for i in range(BATCH_SIZE):
  _,text =next(iter(train_iter))
  batch_of_tokens.append(tokenizer(text))

In [22]:
text=batch_of_tokens[0][0:100]


In [23]:
# Initialize empty lists to store source and target sequences
src_batch, tgt_batch = [], []

# Define the batch size
BATCH_SIZE = 2

# Loop to create batches of source and target sequences
for i in range(BATCH_SIZE):
    # Retrieve the next data point from the training iterator
    _,text = next(iter(train_iter))

    # Generate source and target sequences using the get_sample function
    src_sequence_text, tgt_sequence_text = get_sample(block_size, tokenizer(text))

    # Convert source and target sequences to tokenized vocabulary indices
    src_sequence_indices = vocab(src_sequence_text)
    tgt_sequence_indices = vocab(tgt_sequence_text)

    # Convert the sequences to PyTorch tensors with dtype int64
    src_sequence = torch.tensor(src_sequence_indices, dtype=torch.int64)
    tgt_sequence = torch.tensor(tgt_sequence_indices, dtype=torch.int64)

    # Append the source and target sequences to their respective batches
    src_batch.append(src_sequence)
    tgt_batch.append(tgt_sequence)

    # Print the output for every 2nd sample (adjust as needed)
    print(f"Sample {i}:")
    print("Source Sequence (Text):", src_sequence_text)
    print("Source Sequence (Indices):", src_sequence_indices)
    print("Source Sequence (Shape):", src_sequence.shape)
    print("Target Sequence (Text):", tgt_sequence_text)
    print("Target Sequence (Indices):", tgt_sequence_indices)
    print("Target Sequence (Shape):", tgt_sequence.shape)

Sample 0:
Source Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Source Sequence (Indices): [0, 0, 0]
Source Sequence (Shape): torch.Size([3])
Target Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Target Sequence (Indices): [0, 0, 0]
Target Sequence (Shape): torch.Size([3])
Sample 1:
Source Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Source Sequence (Indices): [0, 0, 0]
Source Sequence (Shape): torch.Size([3])
Target Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Target Sequence (Indices): [0, 0, 0]
Target Sequence (Shape): torch.Size([3])


In [24]:
BLOCK_SIZE=30
def collate_batch(batch):
    src_batch, tgt_batch = [], []
    for _,_textt in batch:
      src_sequence,tgt_sequence=get_sample(BLOCK_SIZE,tokenizer(_textt))
      src_sequence=vocab(src_sequence)
      tgt_sequence=vocab(tgt_sequence)
      src_sequence= torch.tensor(src_sequence, dtype=torch.int64)
      tgt_sequence = torch.tensor(tgt_sequence, dtype=torch.int64)
      src_batch.append(src_sequence)
      tgt_batch.append(tgt_sequence)


    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=False)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=False)

    return src_batch.to(DEVICE), tgt_batch.to(DEVICE)

In [25]:
BATCH_SIZE=1
train_dataloader=DataLoader(train_iter,batch_size=BATCH_SIZE,shuffle=True,collate_fn=collate_batch)
valid_dataloader=DataLoader(valid_iter,batch_size=BATCH_SIZE,shuffle=True,collate_fn=collate_batch)

### Masking

In [28]:
def generate_mask(size):
    mask=torch.tril((torch.ones(size,size))==1)
    mask=mask.float().masked_fill(mask==0,float('-inf')).masked_fill(mask==1,float(0.0))
    return mask

In [29]:
def create_mask(src):
    seq_len=src.shape[0]
    src_mask=generate_mask(seq_len)
    src_padding_mask=(src==vocab("<pad>")).transpose(0,1)
    return src_mask,src_padding_mask